In [1]:
# !pip3 install flask
# !pip3 install pandas
# !pip3 install openpyxl
# !pip3 install geopy

In [2]:
from flask import Flask, render_template_string, request, redirect, url_for, send_file, jsonify,render_template
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from threading import Thread
import pandas as pd
import openpyxl
import threading
import re
import folium
from datetime import datetime
from geopy.distance import geodesic

In [3]:
app = Flask(__name__)
geolocator = Nominatim(user_agent="flask-location-app")

# Load and process data
selected_columns = [8, 9, 11, 12, 17]
df1 = pd.read_excel('q2.xlsx', usecols=selected_columns)
unique_columns = {col: sorted(df1[col].dropna().unique().tolist()) for col in df1.columns}


In [4]:
login_template = """
<!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Jobscape Login</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f9fafc;
            color: #333;
        }
        .login-container {
            max-width: 500px;
            margin: 50px auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }
        h1 {
            color: #5a67d8;
            text-align: center;
        }
        p {
            text-align: center;
            margin-bottom: 20px;
        }
        .option-btn {
            display: inline-block;
            padding: 10px 20px;
            margin: 10px;
            font-size: 1.2em;
            background-color: #5a67d8;
            color: #fff;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        .option-btn:hover {
            background-color: #434190;
        }
    </style>
</head>
<body>
    <div class="login-container">
        <h1>Welcome to Jobscape</h1>
        <p>Select your category:</p>
        <form action="/choose_option" method="POST">
            <!-- Trigger the location request when the "Employee" button is clicked -->
            <button type="submit" name="option" value="Employee" class="option-btn employee-btn" onclick="askForLocation(event)">Employee</button>
            <button type="submit" name="option" value="Employer" class="option-btn employer-btn">Employer</button>
            <button type="submit" name="option" value="Youth Organization" class="option-btn youth-org-btn">Youth Organization</button>
        </form>
    </div>


</body>
</html>

"""


In [5]:
form_template = """
<!doctype html>
<html lang="en">
<head>
    <title>Jobscape Home</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f9fafc;
            color: #333;
        }
        h1 {
            color: #5a67d8;
            margin-bottom: 20px;
        }
        form {
            background-color: #fff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            max-width: 600px;
            margin: 0 auto;
        }
        .dropdown {
            margin-bottom: 15px;
        }
        .dropdown h2 {
            font-size: 1.2em;
            color: #4a5568;
            cursor: pointer;
        }
        .dropdown-content {
            display: none;
            background-color: #f9f9f9;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 4px;
        }
        .location-btn {
            background-color: #38a169;
            color: #fff;
            border: none;
            padding: 10px 20px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 1em;
            margin-top: 10px;
            display: block;
        }
        .location-btn:hover {
            background-color: #2f855a;
        }
        #location-display {
            font-size: 0.9em;
            color: #4a5568;
            margin-top: 10px;
        }
        .search-btn {
            background-color: #5a67d8;
            color: #fff;
            border: none;
            padding: 10px 20px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 1em;
            display: block;
            margin: 20px auto 0;
        }
        .search-btn:hover {
            background-color: #434190;
        }
    </style>
    <script>
        function toggleDropdown(id) {
            var content = document.getElementById(id);
            content.style.display = content.style.display === "block" ? "none" : "block";
        }

        function getLocation() {
            if (navigator.geolocation) {
                navigator.geolocation.getCurrentPosition(showPosition, showError);
            } else {
                alert("Geolocation is not supported by this browser.");
            }
        }

        function showPosition(position) {
            var latitude = position.coords.latitude;
            var longitude = position.coords.longitude;
            document.getElementById("location-display").innerHTML = 
                "Latitude: " + latitude + ", Longitude: " + longitude;
            alert("Location detected: Latitude: " + latitude + ", Longitude: " + longitude);
            document.getElementById("latitude").value = latitude;
            document.getElementById("longitude").value = longitude;
        }

        function showError(error) {
            switch (error.code) {
                case error.PERMISSION_DENIED:
                    alert("User denied the request for Geolocation.");
                    break;
                case error.POSITION_UNAVAILABLE:
                    alert("Location information is unavailable.");
                    break;
                case error.TIMEOUT:
                    alert("The request to get user location timed out.");
                    break;
                case error.UNKNOWN_ERROR:
                    alert("An unknown error occurred.");
                    break;
            }
        }
    </script>
</head>
<body>
    <button class="location-btn" onclick="getLocation()">Choose Location</button>
    <p id="location-display">Your location will appear here.</p>

    <h1>Jobscape</h1>
    <form method="POST" action="/submit">
      {% for column, items in unique_columns.items() %}
        <div class="dropdown">
          <h2 onclick="toggleDropdown('dropdown-{{ column }}')">{{ column }}</h2>
          <div id="dropdown-{{ column }}" class="dropdown-content">
            {% for item in items %}
              <label><input type="checkbox" name="{{ column }}" value="{{ item }}"> {{ item }}</label><br>
            {% endfor %}
          </div>
        </div>
      {% endfor %}
      
      <!-- Hidden fields for latitude and longitude -->
      <input type="hidden" id="latitude" name="latitude">
      <input type="hidden" id="longitude" name="longitude">
    
        <div style="margin-top: 20px;">
        <label for="range">Enter range for job search:</label><br>
        <input type="text" id="range" name="range" placeholder="e.g., 5-10 km" style="width: 100%; padding: 8px; margin-top: 5px;">
        </div>
      <input type="submit" value="Search" class="search-btn">
    </form>
</body>
</html>


"""


In [6]:

employer_email_template = """
<!doctype html>
<html lang="en">
<head>
    <title>Employer Email Verification</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100vh;
            margin: 0;
            background-color: #fffdf9;
        }
        h1 {
            color: #f6ad55;
            margin-bottom: 20px;
        }
        form {
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }
        label, input {
            display: block;
            margin-bottom: 10px;
        }
        input[type="submit"] {
            background-color: #f6ad55;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        input[type="submit"]:hover {
            background-color: #dd9b47;
        }
    </style>
</head>
<body>
    <h1>Enter Business Email for Verification</h1>
    <form action="/verify_email" method="POST">
        <label for="email">Business Email:</label>
        <input type="email" id="email" name="email" required>
        <input type="submit" value="Verify">
    </form>
</body>
</html>
"""

In [7]:

# HTML Template for Employer Actions Page with Hover Tooltip
employer_actions_template = """
<!doctype html>
<html lang="en">
<head>
    <title>Employer Actions</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100vh;
            background-color: #e6fffa;
        }
        h1 { color: #319795; }
        form {
            display: flex;
            gap: 20px;
            margin-top: 20px;
        }
        button {
            padding: 10px 20px;
            background-color: #319795;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            position: relative;
        }
        /* Tooltip styling */
        button:hover::after {
            content: "Please enter job vacancy details";
            position: absolute;
            top: -30px;
            left: 50%;
            transform: translateX(-50%);
            background-color: #333;
            color: #fff;
            padding: 5px 10px;
            border-radius: 4px;
            white-space: nowrap;
            font-size: 12px;
        }
    </style>
</head>
<body>
    <h1>Employer Actions</h1>
    <p>Select an action:</p>
    <form action="/download_csv" method="POST">
        <button type="submit">Download CSV</button>
    </form>
    <form action="/upload_csv" method="POST" enctype="multipart/form-data">
        <input type="file" name="csv_file" accept=".csv" required>
        <button type="submit">Upload CSV</button>
    </form>
</body>
</html>
"""

In [8]:


@app.route('/')
def login():
    return render_template_string(login_template)

@app.route('/choose_option', methods=['POST'])
def choose_option():
    user_option = request.form.get("option")
    if user_option == "Employee":
        return redirect(url_for('home'))
    elif user_option == "Employer":
        return redirect(url_for('employer_email'))
    elif user_option == "Youth Organization":
        return redirect(url_for('youth_organization'))
    else:
        return f"<h1>{user_option} portal coming soon...</h1>"

    # Route for Employer Actions Page
@app.route('/employer_actions')
def employer_actions():
    return render_template_string(employer_actions_template)

# Route to handle CSV download
@app.route('/download_csv', methods=['POST'])
def download_csv():
    csv_data = df1.to_csv(index=False)
    with open('/mnt/data/sample_data.csv', 'w') as file:
        file.write(csv_data)
    return send_file('/mnt/data/sample_data.csv', as_attachment=True, download_name='sample_data.csv')

# Route to handle CSV upload
@app.route('/upload_csv', methods=['POST'])
def upload_csv():
    file = request.files['csv_file']
    if file and file.filename.endswith('.csv'):
        file.save(f"/mnt/data/{file.filename}")
        return "<h1>CSV File Uploaded Successfully</h1>"
    else:
        return "<h1>Invalid File. Please upload a CSV file.</h1>"
    
@app.route('/home')
def home():
    return render_template_string(form_template, unique_columns=unique_columns)

@app.route('/employer_email')
def employer_email():
    return render_template_string(employer_email_template)

@app.route('/verify_email', methods=['POST'])
def verify_email():
    email = request.form.get("email")
    if re.match(r"[^@]+@[^@]+\.[^@]+", email) and email.endswith(".com"):
        return render_template_string(employer_actions_template)
    else:
        return "<h1>Invalid Email. Please try again.</h1>"
@app.route('/map', methods=['GET', 'POST'])
def display_map():
    # Get user inputs
    selected_items = {col: request.form.getlist(str(col)) for col in unique_columns.keys()}
    selected_items = {col: selected_items[col] for col in selected_items if selected_items[col]}
    try:
        user_lat = float(request.args.get("latitude"))
        user_lon = float(request.args.get("longitude"))
        circle_radius = int(request.args.get("range", 5000)) # Default range: 5000 meters
    except (TypeError, ValueError):
        return "Invalid latitude, longitude, or range input.", 400
    print("user_lat and stuff")
    print(user_lat,user_lon,circle_radius)
    circle_radius = int(request.form.get("range", 5000))  # Default to 5 km if range is not provided
    
    # Read the data from the Excel file
    df = pd.read_excel('q2.xlsx')

    # Filter the DataFrame based on selected_items
    for col, values in selected_items.items():
        df = df[df[col].isin(values)]
    # Initialize the map centered on the user's location
    m1 = folium.Map([user_lat, user_lon], zoom_start=13)

    today = datetime.strptime('2024-09-05', '%Y-%m-%d').date()
    # Function to check if the marker is within the circle
    def is_within_circle(marker_location, circle_center, radius):
        return geodesic(marker_location, circle_center).meters <= radius
    # Loop through the filtered DataFrame and add markers
    for vals in df.values:
        marker_location = (vals[4], vals[5])  # Assuming latitude and longitude are in these columns
        if is_within_circle(marker_location, (user_lat, user_lon), circle_radius):
            start = pd.to_datetime(vals[1]).date()
            color = 'blue' if start <= today else 'lightgray'  # Marker color based on date

            info = {
                'Company Registered?': vals[8],
                'Type of Registration': vals[9],
                'Economic Sector': vals[11],
                'Subsector': vals[12],
                'Enterprise Size': vals[17],
                'Phone Number': vals[15],
                'Company Name': vals[10],
            }

            popup_content = f"""
            <div style="font-family: Arial, sans-serif; padding: 10px; background-color: #f9f9f9; border-radius: 5px; border: 2px solid #ccc;">
                <h3 style="color: #333; text-align: center; font-size: 16px; margin-top: 0;">Company Information</h3>
                <ul style="list-style-type: none; padding: 0;">
                    <li><b>Company Registered:</b> {info['Company Registered?']}</li>
                    <li><b>Type of Registration:</b> {info['Type of Registration']}</li>
                    <li><b>Economic Sector:</b> {info['Economic Sector']}</li>
                    <li><b>Subsector:</b> {info['Subsector']}</li>
                    <li><b>Enterprise Size:</b> {info['Enterprise Size']}</li>
                    <li><b>Phone Number:</b> {info['Phone Number']}</li>
                    <li><b>Company Name:</b> {info['Company Name']}</li>
                </ul>
            </div>
            """

            folium.Marker(
                location=marker_location,
                tooltip="Click me!",
                popup=folium.Popup(popup_content, max_width=300),
                icon=folium.Icon(color, icon="cloud")
            ).add_to(m1)

    # Save the map to an HTML file
    map_file = 'templates/map.html'
    print(f"Saving map to {map_file}")
    m1.save(map_file)
    print(f"Saving map to {map_file}")

    return render_template('map.html')


@app.route('/submit', methods=['POST'])
def submit():
    selected_items = {col: request.form.getlist(str(col)) for col in unique_columns.keys()}
    selected_items = {col: selected_items[col] for col in selected_items if selected_items[col]}
    print("selected items")
    print(selected_items)
    # Retrieve latitude and longitude from form
    latitude = request.form.get("latitude")
    longitude = request.form.get("longitude")
    range_input = request.form.get("range")  # Capture the text input value
    if(range_input==''):
        range_input=5000
    if(latitude==''):
        latitude=30.5185291
    if(longitude==''):
        longitude=35.5705103
    print(latitude,longitude,range_input)
    return redirect(url_for('display_map', latitude=latitude, longitude=longitude, range=range_input))


@app.route('/youth_organization')
def youth_organization():
    # Execute the provided DataFrame logic
    df = pd.read_excel('q2.xlsx')
    df = df[df['Do you consider youth (18-29 years) from the local community for employment?'] == 'Yes']
    df['Availability'] = df['start'].apply(
        lambda x: 'Available' if x <= pd.to_datetime("2024-09-05") else 'Pending'
    )
    df_grouped = df.groupby(['Governorat', 'District', 'Availability']).agg(
        Total_Vacancies=('Number of job vacancies:', 'sum')
    ).reset_index()

    available = df_grouped[df_grouped['Availability'] == 'Available']
    pending = df_grouped[df_grouped['Availability'] == 'Pending']

    ycl = available.merge(pending, on=['Governorat'], how='outer').sort_values('Governorat')
    ycl.rename(columns={
        'Total_Vacancies_x': 'Total_Vacancies_Available',
        'Total_Vacancies_y': 'Total_Vacancies_Pending',
        'District': 'District'
    }, inplace=True)
    ycl = ycl[['Governorat', 'District_y', 'Total_Vacancies_Available', 'Total_Vacancies_Pending']].fillna(0)

    # Render the DataFrame to HTML with background gradient
    ycl_html = ycl.style.background_gradient(
        subset=['Total_Vacancies_Available', 'Total_Vacancies_Pending']
    ).to_html()

    # Return the styled DataFrame in a web page
    return f"""
    <!doctype html>
    <html lang="en">
    <head>
        <title>Youth Organization</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 20px;
                padding: 20px;
                background-color: #f9f9f9;
            }}
            table {{
                border-collapse: collapse;
                width: 100%;
                margin-top: 20px;
                background: white;
            }}
            th, td {{
                border: 1px solid #ddd;
                padding: 8px;
                text-align: left;
            }}
            th {{
                background-color: #4a90e2;
                color: white;
            }}
        </style>
    </head>
    <body>
        <h1>Youth Organization Data</h1>
        {ycl_html}
        <a href="/" style="text-decoration: none; color: #4a90e2;">Go Back</a>
    </body>
    </html>
    """


def run_app():
    app.run(host='127.0.0.1', port=1000)

thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:1000
Press CTRL+C to quit
127.0.0.1 - - [19/Nov/2024 16:31:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2024 16:31:15] "POST /choose_option HTTP/1.1" 302 -
127.0.0.1 - - [19/Nov/2024 16:31:15] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2024 16:31:17] "POST /submit HTTP/1.1" 302 -


selected items
{}
30.5185291 35.5705103 5000
user_lat and stuff
30.5185291 35.5705103 5000
Saving map to templates/map.html
Saving map to templates/map.html


127.0.0.1 - - [19/Nov/2024 16:31:18] "GET /map?latitude=30.5185291&longitude=35.5705103&range=5000 HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2024 16:31:27] "POST /choose_option HTTP/1.1" 302 -
127.0.0.1 - - [19/Nov/2024 16:31:31] "GET /youth_organization HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2024 16:31:34] "POST /choose_option HTTP/1.1" 302 -
127.0.0.1 - - [19/Nov/2024 16:31:34] "GET /employer_email HTTP/1.1" 200 -
